<a href="https://colab.research.google.com/github/kobra-kid/Practice/blob/main/Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=a6ceff6e5318f2456fadd7eb1c259787e5a7fc68e50225b50c8eaa91f492e88d
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os


imagePaths = list(paths.list_images('/content/drive/MyDrive/dataset'))
knownEncodings = []
knownNames = []
# перебираем все папки с изображениями
for (i, imagePath) in enumerate(imagePaths):
    # извлекаем имя человека из названия папки
    name = imagePath.split(os.path.sep)[-2]
    # загружаем изображение и конвертируем его из BGR (OpenCV ordering)
    # в dlib ordering (RGB)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #используем библиотеку Face_recognition для обнаружения лиц
    boxes = face_recognition.face_locations(rgb,model='hog')
    # вычисляем эмбеддинги для каждого лица
    encodings = face_recognition.face_encodings(rgb, boxes)
    # добавляем в массивы
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
# сохраним эмбеддинги вместе с их именами в формате словаря
data = {"encodings": knownEncodings, "names": knownNames}
# для сохранения данных в файл используем метод pickle
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

In [ ]:
!pip install aiogram

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
!mkdir /content/temp

In [ ]:
import face_recognition
import pickle
import cv2
import os
from google.colab.patches import cv2_imshow

import asyncio

from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram import F


bot = Bot(token='7019897114:AAH_c7we1ayfDfzGvcysk5l1Lc1Vydely-8')
dp = Dispatcher()
classes = {
    'Frank Iero': 'My Chemical Romance',
    'Gerard Way': 'My Chemical Romance',
    'Mikey Way': 'My Chemical Romance',
    'Ray Toro': 'My Chemical Romance',

    'Anthony Kiedis': 'Red Hot Chili Peppers',
    'Chad Smith': 'Red Hot Chili Peppers',
    'Flea': 'Red Hot Chili Peppers',
    'John Frusciante': 'Red Hot Chili Peppers',

    'josh dun': 'Twenty One Pilots',
    'tyler joseph': 'Twenty One Pilots',
}


@dp.message(Command('start'))
async def start_command(message: types.Message):
    await message.answer('Привет')


@dp.message(F.photo)
async def get_photo_and_result(message: types.Message, bot: Bot):
    image_path = f'temp/{message.photo[1].file_id}.jpg'
    await bot.download(
        message.photo[-1],
        destination=image_path
    )

    await message.reply('Фото загружено!!!')
    names = get_prediction(image_path)

    if len(names) == 0:
      await message.answer('Я не знаю кто это...')
    else:
      # await message.answer(f'Я думаю это {get_prediction(image_path)[0]} из группы {classes[get_prediction(image_path)[0]]}')
      await message.answer(f'Я думаю это {names[0]} из группы {classes[names[0]]}')
      await message.answer_sticker('CAACAgIAAxkBAAEMD0pmOJfZBBFiSE3xfdfPjrheTqLe0wACAwADN16eLWQaln_ddYs3NQQ')


def get_prediction(image_path):
  # путь до файла haarcascade
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  # загружаем файл с известными эмбеддингами
  data = pickle.loads(open('/content/drive/MyDrive/face_enc (1)', "rb").read())
  image = cv2.imread(image_path)
  rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # переводим в серый и ищем лицо
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(gray,
                                      scaleFactor=1.1,
                                      minNeighbors=5,
                                      minSize=(60, 60),
                                      flags=cv2.CASCADE_SCALE_IMAGE)

  # вычисляем эмбеддинг для данного фото
  encodings = face_recognition.face_encodings(rgb)
  names = []
  # проходимся по эмбеддингам
  for encoding in encodings:
      # сравниваем с теми что в файле и значение будет True если ближе всего
      matches = face_recognition.compare_faces(data["encodings"], encoding)
      # для неизвестных кодов
      name = "Unknown"
      # смотрим есть ли совпадение
      if True in matches:
          # находим позиции с тру и пихаем в массив
          matchedIdxs = [i for (i, b) in enumerate(matches) if b]
          counts = {}
          # цикл ждя подсчета распознанных лиц
          for i in matchedIdxs:
              # берем имя из совпавших кодов
              name = data["names"][i]
              # счетчик для этого имени
              counts[name] = counts.get(name, 0) + 1
              # выбираем имя с макс совпадением
              name = max(counts, key=counts.get)
              print(name)
          # добавляем имя в массив
          names.append(name)
  return names


async def main():
    await dp.start_polling(bot)


await main()

Anthony Kiedis
Anthony Kiedis
Anthony Kiedis
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Anthony Kiedis
Anthony Kiedis
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
Gerard Way
G